In [1]:
import pandas as pd   
import numpy as np    
import matplotlib.pyplot as plt 
%matplotlib inline
%config InlineBackend.figure_formats = ['retina']
import seaborn as sns
import time
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
from sklearn.metrics import auc, roc_curve, roc_auc_score, precision_recall_curve
from sklearn.metrics import fbeta_score, cohen_kappa_score
SEED = 42

In [2]:

df = pd.read_csv('modify.csv')
df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,125,33.6,0.627,50,1
1,1,85,66,29,125,26.6,0.351,31,0
2,8,183,64,29,125,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,4,137,40,35,168,43.1,2.288,33,1


In [3]:

X = df.drop('Outcome', axis=1)   # axis=0 for row, axis=1 for column
y = df['Outcome']

# split data to 80:20 ratio for train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=SEED, stratify=y)
print('X_train', X_train.shape)
print('y_train', y_train.shape)
print('X_test', X_test.shape)
print('y_test', y_test.shape)


X_train (614, 8)
y_train (614,)
X_test (154, 8)
y_test (154,)


In [4]:
! pip install sklearn_nature_inspired_algorithms==0.4.3
! pip install NiaPy
! pip install algorithms

In [5]:
def evaluate(model, X_test , y_test,modelname):
  y_pred1=model.predict(X_test)
  precision    = precision_score(y_test, y_pred1)
  recall       = recall_score(y_test, y_pred1)
  f1score      = f1_score(y_test, y_pred1) 
  rocauc       = roc_auc_score(y_test, y_pred1)
  logloss      = log_loss(y_test, y_pred1)
  accuracy     = accuracy_score(y_test, y_pred1)

  df_model = pd.DataFrame({  'model'        : [modelname],
                             'accuracy'     : [accuracy],
                             'precision'    : [precision],
                             'recall'       : [recall],
                             'f1score'      : [f1score],
                             'rocauc'       : [rocauc],
                             'logloss'      : [logloss],
                             'timetaken'    : [time1]   
                                }) 
  return df_model

In [6]:
n_estimators = [int(x) for x in np.linspace(200, 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2,5,10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1,2,4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
param_grid_rt = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(param_grid_rt)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


NATURE INSPIRIED SEARCH


In [7]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from sklearn.ensemble import RandomForestClassifier
time1 = time.time()

clf_1 = RandomForestClassifier()
nia_search = NatureInspiredSearchCV(
    clf_1,
    param_grid_rt,
    cv=3,
    verbose=2,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,
    runs=5,
    n_jobs=-1,
    random_state=42,
    
    # or any number if you want same results on each run
)


nia_search.fit(X_train, y_train)
print("Bat model")

print("Random Forest")

ba_random= nia_search.best_estimator_
model1=evaluate(ba_random,  X_test,  y_test, 'Random Forest')
model1.timetaken[0] = time.time() - time1
print(model1)

Fitting 3 folds for some of the 4320 candidates, which might total in 12960 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    4.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    2.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.4s finished


Bat model
Random Forest
           model  accuracy  precision    recall   f1score    rocauc   logloss  \
0  Random Forest  0.733766   0.632653  0.574074  0.601942  0.697037  9.195482   

     timetaken  
0  2139.266574  
Wall time: 35min 40s


In [8]:
nia_search.best_params_

{'n_estimators': 800,
 'max_features': 'auto',
 'max_depth': 70,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'bootstrap': True}

In [9]:
from sklearn.ensemble import ExtraTreesClassifier
et=ExtraTreesClassifier()
print(et.get_params())

{'bootstrap': False, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [10]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split =  [2,5,10]
# Minimum number of samples required at each leaf node
min_samples_leaf =  [1,2,4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
param_grid_et = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(param_grid_et)

{'n_estimators': [100, 311, 522, 733, 944, 1155, 1366, 1577, 1788, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [11]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
time1 = time.time()
clf_2 = ExtraTreesClassifier()

nia_search2 = NatureInspiredSearchCV(
    clf_2,
    param_grid_et,
    cv=3,
    verbose=2,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,
    runs=5,
    n_jobs=-1,
    random_state=42,
    
    # or any number if you want same results on each run
)


nia_search2.fit(X_train, y_train)
print("Bat model")

print("Extra Tree")

ba_et= nia_search2.best_estimator_
model3=evaluate(ba_et,  X_test,  y_test, 'Extra Tree')
model3.timetaken[0] = time.time() - time1
print(model3)

Fitting 3 folds for some of the 4320 candidates, which might total in 12960 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    4.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    6.7s finished


Bat model
Extra Tree
        model  accuracy  precision    recall   f1score    rocauc  logloss  \
0  Extra Tree  0.727273      0.625  0.555556  0.588235  0.687778  9.41976   

     timetaken  
0  2771.381496  
Wall time: 46min 11s


In [12]:
nia_search2.best_params_

{'n_estimators': 1155,
 'max_features': 'auto',
 'max_depth': None,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'bootstrap': False}

In [13]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
time1 = time.time()
param_grid={'bootstrap': [True],
 'max_depth': [110],
 'max_features': ['auto'],
 'min_samples_leaf':[1],
 'min_samples_split': [5],
 'n_estimators': [800]}
clf_2 = ExtraTreesClassifier()

nia_search2 = NatureInspiredSearchCV(
    clf_2,
    param_grid,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,
    runs=3,
    random_state=42,
    
    # or any number if you want same results on each run
)


nia_search2.fit(X_train, y_train)
print("Bat model")

print("Extra Tree")

ba_et= nia_search2.best_estimator_
model3=evaluate(ba_et,  X_test,  y_test, 'Extra]] Tree')
model3.timetaken[0] = time.time() - time1
print(model3)

Fitting at most 1 candidates
Bat model
Extra Tree
          model  accuracy  precision    recall  f1score    rocauc   logloss  \
0  Extra]] Tree   0.74026   0.652174  0.555556      0.6  0.697778  8.971194   

   timetaken  
0  27.765841  
Wall time: 27.8 s


In [14]:
nia_search2.best_params_

{'bootstrap': True,
 'max_depth': 110,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 800}

In [15]:
from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier()
print(dtc.get_params())

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': 'deprecated', 'random_state': None, 'splitter': 'best'}


In [16]:
class_weight = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]
class_weight.append(None)
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 210, num = 11)]
# Minimum number of samples required to split a node
min_samples_split = [2,5,10,15]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2,4,10]
# Method of selecting samples for training each tree
# Create the random grid
param_grid_dt = {#'class_weight': class_weight,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               }
print(param_grid_dt)

{'max_features': ['auto', 'sqrt'], 'max_depth': [10, 30, 50, 70, 90, 110, 130, 150, 170, 190, 210], 'min_samples_split': [2, 5, 10, 15], 'min_samples_leaf': [2, 4, 10]}


In [17]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
time1 = time.time()

clf_3 = DecisionTreeClassifier()

nia_search3 = NatureInspiredSearchCV(
    clf_3,
    param_grid_dt,
    cv=3,
    verbose=2,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,
    runs=5,
    n_jobs=-1,
    random_state=42,
    
    # or any number if you want same results on each run
)


nia_search3.fit(X_train, y_train)
print("Bat model")

print("Decision Tree")

ba_dt= nia_search3.best_estimator_
model4=evaluate(ba_dt,  X_test,  y_test, 'Decision Tree')
model4.timetaken[0] = time.time() - time1
print(model4)

Fitting 3 folds for some of the 264 candidates, which might total in 792 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished



Fitting 3 folds for each of 1 candidates, totalling 3 fits
Bat model
Decision Tree
           model  accuracy  precision    recall   f1score    rocauc   logloss  \
0  Decision Tree  0.720779   0.617021  0.537037  0.574257  0.678519  9.644038   

   timetaken  
0  10.499385  
Wall time: 10.5 s


In [18]:
nia_search3.best_params_

{'max_features': 'sqrt',
 'max_depth': 110,
 'min_samples_split': 15,
 'min_samples_leaf': 10}

In [19]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
time1 = time.time()
param_grid={
 'max_depth': [115],
 'max_features': ['auto'],
 'min_samples_leaf':[4],
 'min_samples_split': [10],
 }
clf_3 = DecisionTreeClassifier()

nia_search3 = NatureInspiredSearchCV(
    clf_3,
    param_grid,
    cv=3,
    verbose=2,
    algorithm='ba', #  bat algorithm
    population_size=150,
    max_n_gen=200,
    max_stagnating_gen=60,
    runs=5,
    n_jobs=-1,
    random_state=42,
    
    # or any number if you want same results on each run
)


nia_search3.fit(X_train, y_train)
print("Bat model")

print("Extra Tree")

ba_et= nia_search3.best_estimator_
model3=evaluate(ba_et,  X_test,  y_test, 'decision Tree')
model3.timetaken[0] = time.time() - time1
print(model3)

Fitting 3 folds for some of the 1 candidates, which might total in 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished


Bat model
Extra Tree
           model  accuracy  precision    recall   f1score    rocauc   logloss  \
0  decision Tree  0.720779   0.607843  0.574074  0.590476  0.687037  9.644048   

   timetaken  
0    8.34417  
Wall time: 8.35 s


In [20]:
nia_search3.best_params_

{'max_depth': 115,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 10}

In [21]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
print(lr.get_params())

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [22]:
max_iter = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
C= [float(x) for x in np.linspace(start = 1, stop = 20, num = 20)]
solver=['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
class_weight = [int(x) for x in np.linspace(1, 10, num = 10)]
class_weight.append(None)
n_jobs=[int(x) for x in np.linspace(1, 10, num = 10)]
#n_jobs.append(None)
intercept_scaling = [1, 2, 4]
verbose=[0,1,2,3]
dual = [True, False]
multi_class=['auto', 'ovr', 'multinomial']
tol= [0.00001,0.0001,0.001,0.01,0.1,0,1]
# Create the random grid
param_grid_lr = {'max_iter': max_iter,
               #'class_weight': class_weight,
               'solver': solver,
               #'intercept_scaling': intercept_scaling,
               #'dual': dual,
               #'n_jobs' : n_jobs,
               #'verbose' : verbose,
               'C' : C,
               #'tol' : tol
               #'multi_class' : multi_class
              }
print(param_grid_lr)

{'max_iter': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 'C': [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0]}


In [23]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
time1 = time.time()
clf_1 = LogisticRegression()

nia_search1 = NatureInspiredSearchCV(
    clf_1,
    param_grid_lr,
    cv=3,
    verbose=2,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,
    runs=3,
    n_jobs=-1,
    random_state=42,
    
    # or any number if you want same results on each run
)


nia_search1.fit(X_train, y_train)
print("Bat model")

print("Logistic Regression")

ba_lr= nia_search1.best_estimator_
model2=evaluate(ba_lr,  X_test,  y_test, 'Logistic Regression')
model2.timetaken[0] = time.time() - time1
print(model2)

Fitting 3 folds for some of the 1000 candidates, which might total in 3000 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished



Bat model
Logistic Regression
                 model  accuracy  precision    recall  f1score    rocauc  \
0  Logistic Regression  0.701299   0.583333  0.518519  0.54902  0.659259   

     logloss  timetaken  
0  10.316881  69.333562  
Wall time: 1min 9s


In [24]:
nia_search1.best_params_

{'max_iter': 1200, 'solver': 'newton-cg', 'C': 4.0}

In [25]:
from sklearn.neighbors import KNeighborsClassifier
kn=KNeighborsClassifier()
print(kn.get_params())

{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}


In [26]:
leaf_size = [int(x) for x in np.linspace(start = 100, stop = 200, num = 10)]
#n_neighbors=[int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]
#n_neighbors.append(None)
# Number of features to consider at every split
algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
# Maximum number of levels in tree
p = [int(x) for x in np.linspace(1, 20, num = 20)]
param_grid_kn = {'leaf_size': leaf_size,
               'p': p,
               'algorithm': algorithm,
               #'n_neighbors' : n_neighbors,
               }
print(param_grid_kn)

{'leaf_size': [100, 111, 122, 133, 144, 155, 166, 177, 188, 200], 'p': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}


In [27]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
time1 = time.time()
clf_4 = KNeighborsClassifier()

nia_search4 = NatureInspiredSearchCV(
    clf_4,
    param_grid_kn,
    cv=3,
    verbose=2,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,
    runs=3,
    n_jobs=-1,
    random_state=42,
    
    # or any number if you want same results on each run
)


nia_search4.fit(X_train, y_train)
print("Bat model")

print("KNN")

ba_kn= nia_search4.best_estimator_
model4=evaluate(ba_kn , X_test,  y_test, 'KNN')
model4.timetaken[0] = time.time() - time1
print(model4)

Fitting 3 folds for some of the 800 candidates, which might total in 2400 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Bat model
KNN
  model  accuracy  precision    recall   f1score    rocauc    logloss  \
0   KNN  0.688312       0.56  0.518519  0.538462  0.649259  10.765447   

    timetaken  
0  104.544203  
Wall time: 1min 44s


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.2s finished


In [28]:
nia_search4.best_params_

{'leaf_size': 133, 'p': 2, 'algorithm': 'kd_tree'}

In [29]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
time1 = time.time()
clf_4 = KNeighborsClassifier()
param_grid={'algorithm': ['brute'], 'leaf_size': [173], 'p': [1]}
nia_search4 = NatureInspiredSearchCV(
    clf_4,
    param_grid,
    cv=3,
    verbose=2,
    algorithm='ba', #  bat algorithm
    population_size=150,
    max_n_gen=200,
    max_stagnating_gen=60,
    runs=3,
    n_jobs=-1,
    random_state=42,
    
    # or any number if you want same results on each run
)


nia_search4.fit(X_train, y_train)
print("Bat model")

print("KNN")

ba_kn= nia_search4.best_estimator_
model4=evaluate(ba_kn , X_test,  y_test, 'KNN')
model4.timetaken[0] = time.time() - time1
print(model4)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for some of the 1 candidates, which might total in 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished


Bat model
KNN
  model  accuracy  precision    recall   f1score    rocauc    logloss  \
0   KNN  0.694805   0.571429  0.518519  0.543689  0.654259  10.541164   

   timetaken  
0   5.209092  
Wall time: 5.23 s


In [30]:
from sklearn.svm import LinearSVC
lsvc=LinearSVC()
print(lsvc.get_params())

{'C': 1.0, 'class_weight': None, 'dual': True, 'fit_intercept': True, 'intercept_scaling': 1, 'loss': 'squared_hinge', 'max_iter': 1000, 'multi_class': 'ovr', 'penalty': 'l2', 'random_state': None, 'tol': 0.0001, 'verbose': 0}


In [31]:
class_weight = [int(x) for x in np.linspace(start = 10, stop = 110, num = 11)]
class_weight.append(None)
intercept_scaling = [int(x) for x in np.linspace(1, 50, num = 10)]
# Minimum number of samples required to split a node
tol = [0.00001,0.001,0.0001,0.01,0.1,1]
dual = [True, False]
itr=[int(x) for x in np.linspace(100,2000, num = 20)]
# Create the random grid
param_grid_lsvc = {#'class_weight':class_weight,              
               'tol': tol,
               'intercept_scaling': intercept_scaling,
               'dual': dual, 
               'max_iter' : itr,
               }
print(param_grid_lsvc)

{'tol': [1e-05, 0.001, 0.0001, 0.01, 0.1, 1], 'intercept_scaling': [1, 6, 11, 17, 22, 28, 33, 39, 44, 50], 'dual': [True, False], 'max_iter': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000]}


In [32]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
time1 = time.time()
clf_5 = LinearSVC()

nia_search5 = NatureInspiredSearchCV(
    clf_5,
    param_grid_lsvc,
    cv=3,
    verbose=2,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,
    runs=3,
    n_jobs=-1,
    random_state=42,
    
    # or any number if you want same results on each run
)


nia_search5.fit(X_train, y_train)
print("Bat model")

print("LinearSVC")

ba_lsvc= nia_search5.best_estimator_
model5=evaluate(ba_lsvc , X_test,  y_test, 'LinearSVC')
model5.timetaken[0] = time.time() - time1
print(model5)

Fitting 3 folds for some of the 2400 candidates, which might total in 7200 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.1s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.1s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Bat model
LinearSVC
       model  accuracy  precision  recall   f1score  rocauc    logloss  \
0  LinearSVC  0.707792        0.6     0.5  0.545455    0.66  10.092593   

   timetaken  
0  39.277323  
Wall time: 39.3 s


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished


In [33]:
 nia_search5.best_params_

{'tol': 1e-05, 'intercept_scaling': 50, 'dual': False, 'max_iter': 1700}

In [34]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
gnb=GaussianNB()
print(gnb.get_params())

{'priors': None, 'var_smoothing': 1e-09}


In [40]:
priors=[X_train,y_train]
priors.append(None)
var_smoothing=[0.0000000001,0.000001,0.01,0.001,0.00000000000001,1,1e-15,1e-20]
param_grid_gnb = {#'priors':priors,              
               'var_smoothing' : var_smoothing
               }
print(param_grid_gnb)

{'var_smoothing': [1e-10, 1e-06, 0.01, 0.001, 1e-14, 1, 1e-15, 1e-20]}


In [41]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
time1 = time.time()
clf_6 = GaussianNB()

nia_search6 = NatureInspiredSearchCV(
    clf_6,
    param_grid_gnb,
    cv=3,
    verbose=2,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,
    runs=3,
    n_jobs=-1,
    random_state=42,
    
    # or any number if you want same results on each run
)


nia_search6.fit(X_train, y_train)
print("Bat model")

print("GaussianNB")

ba_gnb= nia_search6.best_estimator_
model6=evaluate(ba_gnb , X_test,  y_test, 'GaussianNB')
model6.timetaken[0] = time.time() - time1
print(model6)

Fitting 3 folds for some of the 8 candidates, which might total in 24 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Bat model
GaussianNB
        model  accuracy  precision   recall   f1score    rocauc    logloss  \
0  GaussianNB  0.701299   0.566667  0.62963  0.596491  0.684815  10.316912   

   timetaken  
0   0.915589  
Wall time: 916 ms


In [42]:
 nia_search6.best_estimator_

GaussianNB(priors=None, var_smoothing=1e-20)

In [43]:
from sklearn.svm import SVC
svc=SVC()
print(svc.get_params())

{'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}


In [44]:
degree = [int(x) for x in np.linspace(start = 1, stop = 20, num = 20)]
#class_weight.append(None)
cache_size = [int(x) for x in np.linspace(200, 2000, num = 10)]
# Minimum number of samples required to split a node
tol = [0.00001,0.001,0.0001,0.01,0.1,1]
break_ties = [True, False]
# Create the random grid
param_grid_svc = {'degree': degree,                  
               'tol': tol,
               'cache_size': cache_size,
               'break_ties': break_ties
               }
print(param_grid_svc)

{'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'tol': [1e-05, 0.001, 0.0001, 0.01, 0.1, 1], 'cache_size': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'break_ties': [True, False]}


In [45]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
time1 = time.time()
clf_8 = SVC()

nia_search8 = NatureInspiredSearchCV(
    clf_8,
    param_grid_svc,
    cv=3,
    verbose=2,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,
    runs=3,
    n_jobs=-1,
    random_state=42,
    
    # or any number if you want same results on each run
)


nia_search8.fit(X_train, y_train)
print("Bat model")

print("SVC")

ba_svc= nia_search8.best_estimator_
model8=evaluate(ba_svc , X_test,  y_test, 'SVC')
model8.timetaken[0] = time.time() - time1
print(model8)

Fitting 3 folds for some of the 2400 candidates, which might total in 7200 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Bat model
SVC
  model  accuracy  precision    recall   f1score    rocauc   logloss  \
0   SVC  0.720779   0.648649  0.444444  0.527473  0.657222  9.644012   

   timetaken  
0   27.19558  
Wall time: 27.2 s


In [46]:
 nia_search8.best_params_

{'degree': 4, 'tol': 1e-05, 'cache_size': 200, 'break_ties': False}